## DNN-Hyssop

Author: Justin Tan

Date: Mar. 17 2017

A simple deep neural network for binary classification (signal and $K^* \gamma$ background) on Belle II MC data for $B \rightarrow \rho^0 \gamma$. Separation of $K^* \gamma$ from signal is especially challenging because of the kinematic similarity and identical topology of both decay modes. 

Update 20/03: Added batch normalization, TensorBoard visualization

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time, os

In [2]:
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_integer('learning_rate', 1e-5,
                           """Beta for Adam optimizer.""")
tf.app.flags.DEFINE_integer('batch_size', 128,
                            """Number of training samples subject to SGD per iteration.""")
tf.app.flags.DEFINE_integer('n_epochs', 20,
                            """Number of epochs to train over.""")
tf.app.flags.DEFINE_string('summaries_dir', '/home/ubuntu/radiative/graphs/tensorboard',
                             """Directory to hold tf summary data.""")
tf.app.flags.DEFINE_integer('n_threads', 4,
                            """Number of threads that the queue runner runs in parallel during enqueue op.""")

### Functions for graph construction

In [3]:
def layer_weights(shape):
    # Return weight tensor of given shape using Xavier initialization
    W = tf.get_variable("weights", shape = shape, initializer=tf.contrib.layers.xavier_initializer())
    return W

In [4]:
def layer_biases(shape):
    # Return bias tensor of given shape with small initialized constant value
    b = tf.get_variable("biases", shape = shape, initializer = tf.constant_initializer(0.01))
    return b

In [5]:
def dropout_op(layer, keep_prob):
    # Neurons in given layer have retention probability keep_prob to combat overfitting
    fc_drop = tf.nn.dropout(layer, keep_prob)
    return fc_drop

In [6]:
def hidden_layer_ops(x, shape, name, keep_prob, activation=tf.nn.relu):
    # Add operations to graph to construct hidden layers
    with tf.variable_scope(name) as scope:
        # scope.reuse_variables() # otherwise tf.get_variable() checks that already existing vars are not shared by accident
        weights = layer_weights(shape = shape)
        biases = layer_biases(shape = [shape[1]])
        
        # Apply non-linearity. Default is ReLU
        actv = activation(tf.matmul(x, weights) + biases)
        layer_output = dropout_op(actv, keep_prob)
        
    return layer_output

In [7]:
def readout_ops(x, shape, name, keep_prob, activation=tf.nn.relu):
    # Don't apply non-linearity, dropout on output layer
    with tf.variable_scope(name) as scope:
        weights = layer_weights(shape = shape)
        biases = layer_biases(shape = [shape[1]])
        layer_output = tf.matmul(x, weights) + biases
        
        tf.summary.histogram('readout', layer_output)

    return layer_output

Batch normalization normalizes each scalar feature $x^k$ in each training mini-batch. The normalized output is linearly scaled so that the set of possible transformations contains the identity:
$$ \hat{x}^{(k)} = \frac{x^{(k)} - E[x^{(k)}]}{\sqrt{\mathrm{Var}[x^{(k)}] + \epsilon}}, \quad \quad \theta^k = \gamma^{(k)} \hat{x}^{(k)} + \beta^{(k)} $$
Given a mini-batch $\mathcal{B} = \{x_{1...m}\}$, this is a transformation $\mathrm{BN}(\gamma,\beta) : x_{1,...,m} \rightarrow \theta_{1,...,m}$. The $\gamma, \beta$ tensors become learnable parameters for each layer. When batch-normalizing a network, any layer than previously received $x$ as the input now receives $\mathrm{BN}(x)$.

In [8]:
def BN_layer_ops(x, shape, name, keep_prob, phase, activation=tf.nn.relu):
    # High-level implementation of BN
    with tf.variable_scope(name) as scope:
         # scope.reuse_variables() # otherwise tf.get_variable() checks that already existing vars are not shared by accident
        weights = layer_weights(shape = shape)
        biases = layer_biases(shape = [shape[1]])
        z_BN = tf.matmul(x, weights) + biases
        
        # Place BN transform before or after non-linearity?
        theta_BN = tf.contrib.layers.batch_norm(z_BN, center=True, scale=True, is_training=phase, scope='bn') #, fused = True)
        BN_actv = activation(theta_BN)
        BN_layer_output = dropout_op(BN_actv, keep_prob)

    return BN_layer_output    

In [9]:
def BN_layer_ops_manual(x, shape, name, keep_prob, activation=tf.nn.relu):
    # Low level implementation of BN
    # Apply batch normalization - normalizes a tensor by mean, variance, takes the linear transform of the normalized activation
    with tf.variable_scope(name) as scope:
        # scope.reuse_variables() # otherwise tf.get_variable() checks that already existing vars are not shared by accident
        weights = layer_weights(shape = shape)
        biases = layer_biases(shape = [shape[1]])
        z_BN = tf.matmul(x, weights) + biases
        batch_mean, batch_var = tf.nn.moments(z_BN, [0])
        
        gamma = tf.get_variable(name = 'gamma', shape=[shape[1]], initializer=tf.ones_initializer())
        beta = tf.get_variable(name = 'beta', shape = [shape[1]], initializer = tf.constant_initializer(0.01))
        
        theta_BN = tf.nn.batch_normalization(z_BN, mean = batch_mean, variance = batch_variance, 
                                       offset = beta, scale = gamma, variance_epsilon = 1e-3)
        
        # Apply non-linearity. Default is ReLU
        BN_layer_output = activation(theta_BN)
        
        if name != 'readout': # Don't apply dropout on output layer
            BN_layer_output = dropout_op(layer_output, keep_prob)
            
    return BN_layer_output

In [10]:
def fetch_variables(scope_name):
    # Fetch variables within variable scope_name
    with tf.variable_scope(scope_name, reuse=True):
        W = tf.get_variable("weights")
        b = tf.get_variable("biases")
        return W, b

In [11]:
def onehot(y):
    # Convert labels to one-hot encoding
    onehots = np.zeros((y.shape[0],2))
    for i in range(onehots.shape[0]):
        onehots[i][int(y[i])] = 1
    return onehots 

In [12]:
def binary_onehots(labels, depth=2):
    # Converts signal/background labels to one-hot encoding
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=2)
    return onehot_labels

In [13]:
def cross_entropy(y_true, y_pred):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_true, logits = y_pred))
    tf.summary.scalar('cross_entropy', cross_entropy)
    return cross_entropy

In [14]:
def training_operation(loss_op, learning_rate = 1e-4):
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss_op, name = 'train_op')
    return train_op

In [15]:
def prediction_operation(y_true, y_pred):
    correct_prediction = tf.equal(tf.argmax(y_true,1), tf.argmax(y_pred,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)
    return accuracy

In [16]:
def data_feed_iterator(features, labels, batch_size=128):
    # Takes raw data as numpy arrays and feeds into computation graph
    # Input data has shape [batch_size, n_features]
    while True:
        # Shuffle labels and features: shuf_features = features[idxs], shuf_labels = labels[idxs]
        idxs = np.arange(0, features.shape[0]); np.random.shuffle(idxs)
        random_batch_idx = np.random.choice(idxs, size=batch_size, replace = False)
        
        x_batch = features[random_batch_idx] #.astype('float32')
        labels_batch = labels[random_batch_idx]
        #onehot_labels_batch = tf.one_hot(indices=tf.cast(labels_batch, tf.int32), depth=2)
                
        return [x_batch, labels_batch]

In [17]:
def epoch_shuffle(features, labels):
    # Shuffles features, labels at end of epoch
    rand_idx = np.random.permutation(features.shape[0])
    features = features[rand_idx]
    labels = labels[rand_idx]
    
    return features, labels

#### Placeholder + Hyperparameter Initialization

Start building computation graph by creating nodes for efficient gradient computation.

In [18]:
# Read in data as DataFrame
from sklearn.model_selection import train_test_split
df = pd.read_hdf('/home/ubuntu/radiative/df/rho0/std_norm_sig_cus.h5', 'df')
df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df[df.columns[:-1]], df['labels'], test_size = 0.075, random_state=432)
X_train = df_X_train.values
X_test = df_X_test.values
y_train = onehot(df_y_train.values)
y_test = onehot(df_y_test.values)

In [19]:
n_features = X_train.shape[1] # Number of feature columns in dataset
hidden_layer_nodes = [1024, 512, 1024, 512, 512] # Number of neurons in hidden layers
summaries_dir_custom = '/home/ubuntu/radiative/graphs/tensorboard/rho0/cus'
summaries_dir_cont = '/home/ubuntu/radiative/graphs/tensorboard/rho0/cont'
checkpoints_dir = '/home/ubuntu/radiative/checkpoints/rho0'
n_threads = 4 # number of threads for multi-threaded input queue system
learning_rate = 1e-4
training_steps = 40000
batch_size = 256; dropout_prob = 0.75
v_acc_best = 0; position = 0

In [20]:
# Declare placeholder tensors
x = tf.placeholder(tf.float32, shape = [None, n_features]) # Input features, 58 features per data point
y_true = tf.placeholder(tf.float32, shape = [None, 2]) # Target output classes
keep_prob = tf.placeholder(tf.float32)
training_phase = tf.placeholder(tf.bool)

### Graph Construction

In [21]:
# Strictly initialization of weights, biases - this should be run only once
def build_layers(x, hidden_layer_nodes, keep_prob):
    hidden_1 = add_layer_ops(x, shape = [n_features, hidden_layer_nodes[0]], name = 'hidden1', keep_prob = keep_prob)
    hidden_2 = add_layer_ops(hidden_1, shape = [hidden_layer_nodes[0], hidden_layer_nodes[1]], name = 'hidden2', keep_prob = keep_prob)
    hidden_3 = add_layer_ops(hidden_2, shape = [hidden_layer_nodes[1], hidden_layer_nodes[2]], name = 'hidden3', keep_prob = keep_prob)
    readout = add_layer_ops(hidden_3, shape = [hidden_layer_nodes[2], 2], name = 'readout', keep_prob = 1.0)
    
    return readout

In [22]:
def gradient_ops(y, y_pred):
    train_step_size = 1e-5 # experiment with this

    # Average cross-entropy over the batch dimension as the total loss
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = y_pred))
    train_step = tf.train.AdamOptimizer(train_step_size).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    sess.run(tf.global_variables_initializer())
    
    config = tf.ConfigProto(device_count={"CPU": 4}, inter_op_parallelism_threads=4, intra_op_parallelism_threads=4)

In [23]:
# With batch normalization! Run below cell to compare perfomance
hidden_1 = BN_layer_ops(x, shape = [n_features, hidden_layer_nodes[0]], name = 'BNhidden1', keep_prob = keep_prob, phase = training_phase)
hidden_2 = BN_layer_ops(hidden_1, shape = [hidden_layer_nodes[0], hidden_layer_nodes[1]], name = 'BNhidden2', keep_prob = keep_prob, phase = training_phase)
hidden_3 = BN_layer_ops(hidden_2, shape = [hidden_layer_nodes[1], hidden_layer_nodes[2]], name = 'BNhidden3', keep_prob = keep_prob, phase = training_phase)
hidden_4 = BN_layer_ops(hidden_3, shape = [hidden_layer_nodes[2], hidden_layer_nodes[3]], name = 'BNhidden4', keep_prob = keep_prob, phase = training_phase)
hidden_5 = BN_layer_ops(hidden_4, shape = [hidden_layer_nodes[3], hidden_layer_nodes[4]], name = 'BNhidden5', keep_prob = keep_prob, phase = training_phase)

# Readout layer is the model's unnormalized prediction 
readout = readout_ops(hidden_5, shape = [hidden_layer_nodes[4], 2], name = 'readout', keep_prob = 1.0)

In [ ]:
hidden_1 = hidden_layer_ops(x, shape = [n_features, hidden_layer_nodes[0]], name = 'hidden1', keep_prob = keep_prob)
hidden_2 = hidden_layer_ops(hidden_1, shape = [hidden_layer_nodes[0], hidden_layer_nodes[1]], name = 'hidden2', keep_prob = keep_prob)
hidden_3 = hidden_layer_ops(hidden_2, shape = [hidden_layer_nodes[1], hidden_layer_nodes[2]], name = 'hidden3', keep_prob = keep_prob)
hidden_4 = hidden_layer_ops(hidden_3, shape = [hidden_layer_nodes[2], hidden_layer_nodes[3]], name = 'hidden4', keep_prob = keep_prob)
hidden_5 = hidden_layer_ops(hidden_4, shape = [hidden_layer_nodes[3], hidden_layer_nodes[4]], name = 'hidden5', keep_prob = keep_prob)

# Readout layer is the model's unnormalized prediction 
readout = readout_ops(hidden_5, shape = [hidden_layer_nodes[4], 2], name = 'readout', keep_prob = 1.0)

In [24]:
# Run these operations only during inference
prediction_op = tf.nn.softmax(readout)
classification_op = tf.argmax(prediction_op, 1)

In [25]:
# Can retrieve initalized network parameters as (weights, biases)
params4 = fetch_variables('BNhidden4')
[param.get_shape() for param in params4]

[TensorShape([Dimension(1024), Dimension(512)]), TensorShape([Dimension(512)])]

The last output layer can be interpreted as the class scores.

Define the cross-entropy loss function: with network output $\hat{y} = g(\mathbf{w} \cdot \mathbf{x} + \mathbf{b})$ the observed (true) probabilities are given as $ p \in \{y, 1-y\}$, and the predicted probability is $q \in \{\hat{y}, 1- \hat{y} \}$; cross-entropy is a measure of the similarity between $p$ and $q$:

$$ H(p,q) = - \sum_i p_i \log(q_i) $$

The typical loss function is given by the average over all cross-entropies in the sample:

$$ L(\mathbf{w}) = \frac{1}{N}\sum_i H_i(p,q) $$

Reduce overfitting by applying dropout on each layer. Units and their connections are automatically dropped from the network during training to aid generalization during test time. The full unthinned network is used to evaluate the test data. For input units the optimal probability of retention is 1, for hidden layers we need to experiment...

In [26]:
# Add operations for loss minimization
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
cross_entropy_op = cross_entropy(y_true = y_true, y_pred = readout)

with tf.control_dependencies(update_ops):
    # Ensures that we execute the update_ops before performing the train_step
    train_op = training_operation(cross_entropy_op, learning_rate = learning_rate)
    
accuracy = prediction_operation(y_true = y_true, y_pred = readout)
# Save checkpoints
saver = tf.train.Saver()
# Merge all the summaries and write them out to /tmp/mnist_logs (by default)
merge_op = tf.summary.merge_all()
# Write summary data to disk for TensorBoard visualization
train_writer = tf.summary.FileWriter(summaries_dir_custom + '/train', graph = tf.get_default_graph())
test_writer = tf.summary.FileWriter(summaries_dir_custom + '/test')

This part adds the training operations to the graph needed to minimize the loss via some form of stochastic gradient descent. Training proceeds over N iterations.

In [27]:
with tf.Session() as sess: # automatically closes upon exit
    # Initialize variables
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    start_time = time.time()
    
    # Launch the graph and train on randomly selected subset of training data
    for step in range(training_steps):            
        # Fetch input data from specified stream
        if ((X_train.shape[0] - position) > batch_size):
            batch_train = [X_train[position:position+batch_size], y_train[position:position+batch_size]]
            position += batch_size
        else:
            # Epoch completed, reshuffle training data, labels
            batch_train = [X_train[position:], y_train[position:]]
            position = 0
            X_train, y_train = epoch_shuffle(X_train, y_train)
            print("End of epoch.")
            
        # batch_test = data_feed_iterator(X_train.values[position:batch_size], y_train.values[position:batch_size], batch_size=batch_size)
        feed_dict_train = {x: batch_train[0], y_true: batch_train[1], keep_prob: dropout_prob, training_phase: True}
        
        # Inject data into Tensors in computation graph - optimize this!
        # train_op.run(feed_dict = feed_dict_train)
        t_step, t_error = sess.run([train_op, cross_entropy_op], feed_dict = feed_dict_train)
    
        if step % 20 == 0:
            # Record summaries
            summary = sess.run(merge_op, feed_dict = feed_dict_train)
            train_writer.add_summary(summary, step)

        if step % 200 == 0:
            batch_val = data_feed_iterator(X_test, y_test, batch_size=batch_size)
            improved = ''
            # t_acc = accuracy.eval(feed_dict = {x: batch_test[0], y_true: batch_test[1], keep_prob = 1.0})
            # v_acc = accuracy.eval(feed_dict = {x: batch_val[0], y_true: batch_val[1], keep_prob = 1.0})
            t_acc = sess.run(accuracy, feed_dict = {x: batch_train[0], y_true: batch_train[1], keep_prob: 1.0, training_phase: False})
            v_acc, v_summary = sess.run([accuracy, merge_op], feed_dict = {x: batch_val[0], y_true: batch_val[1], keep_prob: 1.0, training_phase: False})
            test_writer.add_summary(v_summary, step)
            
            if v_acc > v_acc_best:
                v_acc_best = v_acc
                improved = '*'
                
            delta_t = time.time() - start_time
            print("Step: %d\t|\tTraining accuracy: %g\t|\tValidation Accuracy: %g (%.3f s) %s"
                  %(step, t_acc, v_acc, delta_t, improved))

    # Training complete
    save_path = saver.save(sess, os.path.join(checkpoints_dir, 'rho0_hyssop.ckpt'), global_step = step)
    print("Model saved in file: %s" % save_path)
    
    final_train_accuracy = accuracy.eval(feed_dict = {x: X_train, y_true: y_train, keep_prob: 1.0, training_phase: False})
    final_val_accuracy = accuracy.eval(feed_dict = {x: X_test, y_true: y_test, keep_prob: 1.0, training_phase: False})

    delta_t = time.time() - start_time
    NN_meta = "Architecture: %s | Epochs: %d, Duration: %.2f s" %(str(hidden_layer_nodes), training_steps*batch_size/(X_train.shape[0]), delta_t)    
    
    print("Training Complete. Epochs: %d\n" %(training_steps*batch_size/(X_train.shape[0])))
    print("Train accuracy: %g\nValidation accuracy: %g" %(final_train_accuracy, final_val_accuracy))
    print("Duration: %g s" %(delta_t))

Step: 0	|	Training accuracy: 0.546875	|	Validation Accuracy: 0.476562 (0.607 s) *
Step: 200	|	Training accuracy: 0.722656	|	Validation Accuracy: 0.722656 (14.585 s) *
Step: 400	|	Training accuracy: 0.796875	|	Validation Accuracy: 0.824219 (28.359 s) *
Step: 600	|	Training accuracy: 0.785156	|	Validation Accuracy: 0.816406 (42.131 s) 
Step: 800	|	Training accuracy: 0.757812	|	Validation Accuracy: 0.8125 (56.322 s) 
Step: 1000	|	Training accuracy: 0.753906	|	Validation Accuracy: 0.789062 (70.366 s) 
Step: 1200	|	Training accuracy: 0.734375	|	Validation Accuracy: 0.828125 (84.371 s) *
Step: 1400	|	Training accuracy: 0.785156	|	Validation Accuracy: 0.785156 (98.263 s) 
End of epoch.
Step: 1600	|	Training accuracy: 0.757812	|	Validation Accuracy: 0.761719 (112.259 s) 
Step: 1800	|	Training accuracy: 0.800781	|	Validation Accuracy: 0.800781 (126.237 s) 
Step: 2000	|	Training accuracy: 0.789062	|	Validation Accuracy: 0.8125 (140.340 s) 
Step: 2200	|	Training accuracy: 0.785156	|	Validation Ac

#### Making Predictions
Classification on a new instance is given by the softmax of the output of the final readout layer.

In [ ]:
# Load saved graph metadata in current default graph, restore in session
saver = tf.train.import_meta_graph(os.path.join(checkpoints_dir, 'simple_model-39999.meta'))
graph = tf.get_default_graph()
checkpoint_to_restore = 'simple_model-39999'

In [ ]:
with tf.Session() as sess: 
    start_time = time.time()
    
    # Restore the trained model
    saver.restore(sess, os.path.join(checkpoints_dir, checkpoint_to_restore))
    print("Model restored.")

    feed_dict_train = {x: X_train, keep_prob: 1.0, training_phase: False}
    feed_dict_test = {x: X_test, keep_prob: 1.0, training_phase: False}
    
    # Make predictions using the trained model
    network_output_train, classifications_train = sess.run([prediction_op, classification_op], feed_dict = feed_dict_train)
    network_output_test, classifications_test = sess.run([prediction_op, classification_op], feed_dict = feed_dict_test)
    
    delta_t = time.time() - start_time
    print("Inference complete. Duration: %g s" %(delta_t))

In [ ]:
# Save neural network output to disk
np.save('/home/ubuntu/radiative/df/rho0/arrays/n_train.npy', network_output_train)
np.save('/home/ubuntu/radiative/df/rho0/arrays/y_train.npy', y_train)
np.save('/home/ubuntu/radiative/df/rho0/arrays/n_test.npy', network_output_test)
np.save('/home/ubuntu/radiative/df/rho0/arrays/y_test.npy', y_test)